In [2]:
# CS340 Project Two - Rikuto Ota

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import base64
import os

# Configure the plotting routines
import numpy as np
import matplotlib.pyplot as plt

# Import the CRUD module
# (This file connects to MongoDB and handles read/write operations)
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Replace with your MongoDB username and password
username = "aacuser"
password = "YourPasswordHere"

# Connect to MongoDB using the CRUD module
db = AnimalShelter(username, password)

# Retrieve all animal records from the database
df = pd.DataFrame.from_records(db.read({}))

# MongoDB includes '_id' by default, but we remove it to avoid problems
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Load and encode logo image
# (This image will be shown on top of the dashboard)
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Start Dash app
app = JupyterDash(__name__)

# Page layout (Header, filter options, data table, charts)
app.layout = html.Div([
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '60px'}),
        html.H2("CS-340 Dashboard - Rikuto Ota")
    ], style={'textAlign': 'center'}),

    html.Hr(),

    # Radio buttons for rescue type filters
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'inline-block', 'margin-right': '20px'},
        style={'padding': '10px'}
    ),

    # Table showing animal data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        page_size=10
    ),

    # Graph and map side by side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Highlight selected columns in the table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Update the table based on selected rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'water':
        records = db.read_water_rescue()
    elif filter_type == 'mountain':
        records = db.read_mountain_rescue()
    elif filter_type == 'disaster':
        records = db.read_disaster_rescue()
    else:
        records = db.read({})

    dff = pd.DataFrame.from_records(records)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')

# Show bar chart of breeds
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return html.P("No data to display.")

    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']

    return [
        dcc.Graph(figure=px.bar(breed_counts, x='breed', y='count', title='Breed Count'))
    ]

# Show selected animal on the map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or not index:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    try:
        lat = float(dff.iloc[row]['location_lat'])
        lon = float(dff.iloc[row]['location_long'])
    except:
        return []

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row]['name'])])
            ])
        ])
    ]

# Run the app
app.run_server(mode='inline', debug=True, port=8052)
